In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '23224'

In [5]:
task =  "Consider the first week of August 2020 - find 3 pitchers who's curveballs were most similar to Max Scherzer's."
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

{'Orchestrate': {'next': 'Plan'}}
----
Distance to neareast plan: 0.5457632541656494
Formulating a new plan based on User input
*****function_detail*******
Text between the <playerid_lookup_documentation></playerid_lookup_documentation> tags is documentation for the playerid_lookup function.  Consult this section to confirm which attributes to pass into the playerid_lookup function.
<playerid_lookup_documentation>
# Player ID Lookup

## Single Player Lookup

`playerid_lookup(last, first=None, fuzzy=False)`

Look up a player's MLBAM, Retrosheet, FanGraphs, and Baseball Reference ID by name.

## Arguments
`last:` String. The player's last name. Case insensitive.

`first:` String. Optional. The player's first name. Case insensitive.

`fuzzy:` Boolean. Optional. Search for inexact name matches, the 5 closest will be returned.

Providing last name only will return all available id data for players with that last name (this will return several rows for a common last name like Jones, for exam

In [ ]:
task = """
make sure the plan follows this general flow:
1) create an average vector for each pitcher's curveball.
2) Train a knn model on this data
3) Use this model to find the 3 pitchers with the most similar curveball
"""
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'approved'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'yes'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'yes, thank you'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)